# Band Structure Calculation using Tight Binding Theory About Any (basis, hopping parameter)


In [1]:

import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

def cal_deltas(lattice, basis, t):
    num_basis = len(basis)
    max_n = max(len(t[i][j]) for i in range(num_basis) for j in range(num_basis))
    max_range = (max_n + 1) ** 2

    deltas = [[[] for _ in range(num_basis)] for _ in range(num_basis)]
    for i in range(num_basis):
        for j in range(num_basis):
            distance_groups = defaultdict(list)
            for m in range(-max_range, max_range + 1):
                for n in range(-max_range, max_range + 1):
                    point = basis[j] + (m * lattice[0] + n * lattice[1])
                    delta = point - basis[i]
                    length = np.linalg.norm(delta)
                    distance_groups[round(length, 5)].append(delta)

            sorted_distances = sorted(distance_groups.keys())
            for k in range(len(t[i][j])):
                deltas[i][j].append(np.array(distance_groups[sorted_distances[k]], dtype=object))
    
    return deltas

def hamiltonian(lattice, basis, t, kv, deltas):
    num_basis = len(basis)
    H = np.zeros((num_basis, num_basis), dtype=complex)

    for i in range(num_basis):
        for j in range(num_basis):
            for k in range(len(t[i][j])):
                for d in deltas[i][j][k]:
                    if k == 0 and i == j:
                        H[i, j] += t[i][j][k]
                    else:
                        H[i, j] += t[i][j][k] * np.exp(1j * np.dot(kv, d))
    return H

def E(lattice, basis, t, kv, deltas):
    H = hamiltonian(lattice, basis, t, kv, deltas)
    E_values, _ = np.linalg.eigh(H)
    return E_values[0], E_values[1]

# TB Model Optimization Code
# minimize method can change
# Weight X

In [ ]:

import numpy as np
from scipy.optimize import minimize

def objective_function(t_vector, lattice, basis, kv_points, dft_energies):

    t = [[t_vector[0:3], t_vector[3:6]],
        [t_vector[3:6], t_vector[6:9]]]

    delta = cal_deltas(lattice, basis, t)

    error = 0.0
    for kv, dft_energy in zip(kv_points, dft_energies):
        E1, E2 = E(lattice, basis, t, kv, delta)
        error += (E1 - dft_energy[0])**2 + (E2 - dft_energy[1])**2

    return error

def optimize_t_vector(lattice, basis, kv_points, dft_energies, initial_t_vector=None):
    if initial_t_vector is None:
        initial_t_vector = np.random.normal(-2, 3, 9)

    print("Initial t_vector:", initial_t_vector)

    result = minimize(objective_function, initial_t_vector, args=(lattice, basis, kv_points, dft_energies), 
                      method='L-BFGS-B')

    optimized_t_vector = result.x

    optimized_t = [
        [optimized_t_vector[0:3], optimized_t_vector[3:6]],
        [optimized_t_vector[3:6], optimized_t_vector[6:9]]
    ]

    return optimized_t

# Weight O

In [ ]:
import numpy as np
from scipy.optimize import minimize

def calculate_weight(kv, reference_kv, weight_factor):
    distance = np.linalg.norm(kv - reference_kv)
    weight = np.exp(-weight_factor * distance) 
    return weight

def objective_function(t_vector, lattice, basis, kv_points, dft_energies, reference_kv, weight_factor):
    t = [[t_vector[0:3], t_vector[3:6]],
         [t_vector[3:6], t_vector[6:9]]]

    delta = cal_deltas(lattice, basis, t)

    error = 0.0
    for kv, dft_energy in zip(kv_points, dft_energies):
        E1, E2 = E(lattice, basis, t, kv, delta)
        weight = calculate_weight(kv, reference_kv, weight_factor)
        error += weight * ((E1 - dft_energy[0])**2 + (E2 - dft_energy[1])**2)

    return error

def optimize_t_vector(lattice, basis, kv_points, dft_energies, initial_t_vector=None):
    if initial_t_vector is None:
        initial_t_vector = np.random.normal(-2, 3, 9)

    print("Initial t_vector:", initial_t_vector)

    result = minimize(objective_function, initial_t_vector, args=(lattice, basis, kv_points, dft_energies), 
                      method='L-BFGS-B')

    optimized_t_vector = result.x

    optimized_t = [
        [optimized_t_vector[0:3], optimized_t_vector[3:6]],
        [optimized_t_vector[3:6], optimized_t_vector[6:9]]
    ]

    return optimized_t

# Graphene

### Limit the analysis to k-points where the conduction band and valence band are clearly separated in the band structure calculation results obtained using Quantum Espresso

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error, r2_score

file_path = 'graphene.band.dat'

with open(file_path, 'r') as file:
    lines = file.readlines()

k_points = []
energy_bands = []

for i in range(1, len(lines), 3):
    k_point = list(map(float, lines[i].split()))
    energy_band_10 = list(map(float, lines[i + 1].split()))
    energy_band_6 = list(map(float, lines[i + 2].split()))
    
    k_points.append(k_point)
    energy_bands.append(energy_band_10 + energy_band_6)

df = pd.DataFrame(energy_bands, columns=[f'Band_{i+1}' for i in range(16)])
df.insert(0, 'kx', [kp[0] for kp in k_points])
df.insert(1, 'ky', [kp[1] for kp in k_points])
df.insert(2, 'kz', [kp[2] for kp in k_points])

df_subset = df.iloc[11:49].reset_index(drop=True)

a = 2.46
lattice = a * np.array([[1, 0],
                        [-1/2, np.sqrt(3)/2]])
basis = np.array([2/3*lattice[0]+1/3*lattice[1], 1/3*lattice[0]+2/3*lattice[1]])

g = 2 * np.pi * np.linalg.inv(lattice).T

fermi_level = -3.4482

kv_points = df_subset[['kx', 'ky']].values*2*np.pi/a   # (kx, ky)를 numpy array로 변환
dft_energies = df_subset[['Band_4', 'Band_5']].values
dft_energies_1 = df_subset['Band_4'].values
dft_energies_2 = df_subset['Band_5'].values

def objective_function(t_vector, lattice, basis, kv_points, dft_energies):
    epsilon = -3.4482 + 3*t_vector[0] - 6*t_vector[1]
    
    t = [[[epsilon, t_vector[0], t_vector[1]], t_vector[2:4]],
        [t_vector[2:4],[epsilon, t_vector[0], t_vector[1]]]]

    delta = cal_deltas(lattice, basis, t)

    error = 0.0
    for kv, dft_energy in zip(kv_points, dft_energies):
        E1, E2 = E(lattice, basis, t, kv, delta)
        error += (E1 - dft_energy[0])**2 + (E2 - dft_energy[1])**2

    return error

initial_t_vector = np.random.normal(-2,3,4)

print(initial_t_vector)

result = minimize(objective_function, initial_t_vector, args=(lattice, basis, kv_points, dft_energies), 
                  method='L-BFGS-B')

optimized_t_vector = result.x

epsilon = -3.4482 + 3*optimized_t_vector[0] - 6*optimized_t_vector[1]

optimized_t = [
    [[epsilon, optimized_t_vector[0], optimized_t_vector[1]], optimized_t_vector[2:4]],
    [optimized_t_vector[2:4], [epsilon, optimized_t_vector[0], optimized_t_vector[1]]]
]

print(f'최적화된 t 매개변수:\n{optimized_t}')

t = optimized_t

deltas = cal_deltas(lattice, basis, t)

tb_energies_1 = []
tb_energies_2 = []

for kx, ky in kv_points:
    kv = np.array([kx, ky])

    e1, e2 = E(lattice, basis, t, kv, deltas)
    tb_energies_1.append(e1)
    tb_energies_2.append(e2)

plt.figure(figsize=(10, 6))
plt.plot(tb_energies_1, label='E1_TB')
plt.plot(tb_energies_2, label='E2_TB')
plt.axhline(y=-3.4482, color='green', linestyle='--', label='Fermi Level')
for i in range(3, 5):
    plt.plot(df_subset.index, df_subset[f'Band_{i+1}'], label=f'E{i-2}_QE')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel('k-point index')
plt.ylabel('Energy (eV)')
plt.title('Energy Bands for TB Model vs QE Model')
plt.grid(True)
plt.tight_layout()
plt.show()

rmse_1 = np.sqrt(mean_squared_error(dft_energies_1, tb_energies_1))
rmse_2 = np.sqrt(mean_squared_error(dft_energies_2, tb_energies_2))

r2_1 = r2_score(dft_energies_1, tb_energies_1)
r2_2 = r2_score(dft_energies_2, tb_energies_2)

print(f'Band 1: RMSE = {rmse_1}, R^2 = {r2_1}')
print(f'Band 2: RMSE = {rmse_2}, R^2 = {r2_2}')
print(f'Average RMSE = {(rmse_1+rmse_2)/2}, Average R^2 = {(r2_1+r2_2)/2}')

# h-BN

### Limit the analysis to k-points where the conduction band and valence band are clearly separated in the band structure calculation results obtained using Quantum Espresso.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error, r2_score

file_path = 'BN.band.dat'

# DFT Band data
with open(file_path, 'r') as file:
    lines = file.readlines()

k_points = []
energy_bands = []

for i in range(1, len(lines), 3):
    k_point = list(map(float, lines[i].split()))
    energy_band_10 = list(map(float, lines[i + 1].split()))
    energy_band_6 = list(map(float, lines[i + 2].split()))
    
    k_points.append(k_point)
    energy_bands.append(energy_band_10 + energy_band_6)

df = pd.DataFrame(energy_bands, columns=[f'Band_{i+1}' for i in range(16)])
df.insert(0, 'kx', [kp[0] for kp in k_points])
df.insert(1, 'ky', [kp[1] for kp in k_points])
df.insert(2, 'kz', [kp[2] for kp in k_points])

df_subset = df.iloc[11:48].reset_index(drop=True)

kv_points = df_subset[['kx', 'ky']].values*2*np.pi/a   # (kx, ky)를 numpy array로 변환
dft_energies = df_subset[['Band_4', 'Band_5']].values
dft_energies_1 = df_subset['Band_4'].values
dft_energies_2 = df_subset['Band_5'].values

# Hexagonal Boron Nitride definition

a = 2.50

lattice = a * np.array([
                       [1,0],
                       [-1/2, np.sqrt(3)/2]])

basis = np.array([2/3*lattice[0]+1/3*lattice[1], 1/3*lattice[0]+2/3*lattice[1]])

g = 2 * np.pi * np.linalg.inv(lattice).T

fermi_level = -2.5382

kv_points = df_subset[['kx', 'ky']].values*2*np.pi/a   # (kx, ky)를 numpy array로 변환
dft_energies = df_subset[['Band_4', 'Band_5']].values
dft_energies_1 = df_subset['Band_4'].values
dft_energies_2 = df_subset['Band_5'].values

# Optimization


def objective_function(t_vector, lattice, basis, kv_points, dft_energies):

    t = [[t_vector[0:2], t_vector[2:4]],
        [t_vector[2:4], t_vector[4:7]]]

    delta = cal_deltas(lattice, basis, t)

    error = 0.0
    for kv, dft_energy in zip(kv_points, dft_energies):
        E1, E2 = E(lattice, basis, t, kv, delta)
        error += (E1 - dft_energy[0])**2 + (E2 - dft_energy[1])**2

    return error

initial_t_vector = np.random.normal(-2,3,7)

print(initial_t_vector)

result = minimize(objective_function, initial_t_vector, args=(lattice, basis, kv_points, dft_energies), 
                  method='L-BFGS-B')

optimized_t_vector = result.x

optimized_t = [
    [optimized_t_vector[0:2], optimized_t_vector[2:4]],
    [optimized_t_vector[2:4], optimized_t_vector[4:7]]
]

print(f'최적화된 t 매개변수:\n{optimized_t}')

t = optimized_t

deltas = cal_deltas(lattice, basis, t)

tb_energies_1 = []
tb_energies_2 = []

for kx, ky in kv_points:
    kv = np.array([kx, ky])

    e1, e2 = E(lattice, basis, t, kv, deltas)
    tb_energies_1.append(e1)
    tb_energies_2.append(e2)

plt.figure(figsize=(10, 6))
plt.plot(tb_energies_1, label='E1_TB')
plt.plot(tb_energies_2, label='E2_TB')
plt.axhline(y=-3.4482, color='green', linestyle='--', label='Fermi Level')
for i in range(3, 5):
    plt.plot(df_subset.index, df_subset[f'Band_{i+1}'], label=f'E{i-2}_QE')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel('k-point index')
plt.ylabel('Energy (eV)')
plt.title('Energy Bands for TB Model vs QE Model')
plt.grid(True)
plt.tight_layout()
plt.show()

rmse_1 = np.sqrt(mean_squared_error(dft_energies_1, tb_energies_1))
rmse_2 = np.sqrt(mean_squared_error(dft_energies_2, tb_energies_2))

r2_1 = r2_score(dft_energies_1, tb_energies_1)
r2_2 = r2_score(dft_energies_2, tb_energies_2)

print(f'Band 1: RMSE = {rmse_1}, R^2 = {r2_1}')
print(f'Band 2: RMSE = {rmse_2}, R^2 = {r2_2}')
print(f'Average RMSE = {(rmse_1+rmse_2)/2}, Average R^2 = {(r2_1+r2_2)/2}')